<h2>Zadání 6. Analýza projektů (gql_ug + gql_projects)</h2>

- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

<b>Instalce potřebných knihoven</b>

In [139]:
%pip install pandas aiohttp plotly.express asyncio nbformat

Note: you may need to restart the kernel to use updated packages.


In [140]:
import aiohttp
import asyncio
import json
import plotly.express as px
import pandas as pd
from itertools import product
from functools import reduce

<b>Funkce pro získání tokenu</b>

In [141]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            tokenJson = await resp.json()
    return tokenJson.get("token", None)

<b>Funkce pro definici GraphQL dotazu</b>

In [142]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

<b>Pomocné funkce pro zpracování dat</b>

In [143]:
def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

<b>Přihlašovací údaje</b>

In [144]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

<b>GraphQL dotaz</b>

Where (např. podle validity)

In [145]:
queryStr = """
{
  projectPage(where: { valid: { _eq: true } }) {
    id
    name
    startdate
    enddate
    valid
    projectType {
      id
      name
    }
    milestones {
      id
    }
    group {
      id
      name
    }
  }
}
"""

<b>Mapování atributů</b>

In [146]:
mappers = {
    "projectID": "id",
    "projectName": "name",
    "startDate": "startdate",
    "endDate": "enddate",
    "validity": "valid",
    "projectTypeID": "projectType.id",
    "projectType": "projectType.name",
    "milestonesCount": "milestonesCount",
    "groupID": "group.id",
    "groupName": "group.name",
}

<b>Tvorba tabulky</b>

In [147]:
def createTable(tableData):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    df = pd.DataFrame(tableData)

    print(df)

<b>Tvorba grafů</b>

In [148]:
def createGraphs(graphData):
    df = pd.DataFrame(graphData)

    fig_sunburst = px.sunburst(
        df,
        path=['groupName', 'projectName', 'milestonesCount'],
        values='milestonesCount', 
        title='Project Distribution by Group, Project Name, and Milestones Count'
    )

    fig_sunburst.update_layout(
        width=800,  
        height=800 
    )

    df_bar = df.groupby(['groupName']).size().reset_index(name='counts')

    fig_bar = px.bar(
        df_bar,
        x='groupName',
        y='counts',
        barmode='group',
        title='Number of Projects by Group',
        labels={'groupName': 'Group Name', 'counts': 'Number of Projects'}
    )

    fig_bar.update_layout(
        yaxis=dict(
            tickmode='linear',
            dtick=1
        )
    )

    fig_sunburst.show()
    fig_bar.show()

<b>Asynchronní funkce pro celý proces</b>

In [149]:
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("projectPage", None)
    
    resultMapped = list(map(lambda project: {**project, "milestonesCount": len(project.get("milestones", []))}, result))
    flatData = list(flatten(resultMapped, {}, mappers))
    
    createTable(flatData)
    createGraphs(flatData)

    return flatData

async def main():
    pipeResult = await fullPipe()
    with open('resultNotebook.json', "w", encoding='utf-8') as outputFile:
        json.dump(pipeResult, outputFile)

await main()

                              projectID                   projectName            startDate              endDate  validity  milestonesCount                         projectTypeID projectType                               groupID groupName
0  43dd2ff1-5c17-42a5-ba36-8b30e2a243bb  Nukleární reaktor pro budovy  2023-01-01T17:27:12  2025-12-31T17:27:12      True                2  a825d8e1-2e60-4884-afdb-25642db581d8        GAČR  2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni


<h2>Falešné data</h2>

<b>Zpracování falešných dat</b>

In [150]:
def createTableRow(project):
    row = {}
    row["projectID"] = project["id"]
    row["projectName"] = project["name"]
    row["startDate"] = project["startdate"]
    row["endDate"] = project["enddate"]
    row["validity"] = project["valid"]

    row["projectTypeID"] = project["projectType"]["id"]
    row["projectType"] = project["projectType"]["name"]

    row["milestonesCount"] = len(project['milestones'])

    row["groupID"] = project["group"]["id"]
    row["groupName"] = project["group"]["name"]
    
    return row


with open('dataFake.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)


sourceTable = []


for project in data["data"]["projectPage"]:
    row = createTableRow(project)
    sourceTable.append(row)


with open('resultFake.json', "w", encoding='utf-8') as outputFile:
    json.dump(sourceTable, outputFile)

<b>Vytvoření tabulky</b>

In [151]:
with open("resultFake.json", "r") as file:
    data = json.load(file)

createTable(data)

                               projectID                     projectName                   startDate                     endDate  validity                         projectTypeID projectType  milestonesCount                               groupID   groupName
0   d765d9ca-0eb0-4c9b-977b-52b59d18a6a1          Inteligentní osvětlení  2023-11-02T08:38:52.038979  2025-10-02T08:38:52.038982      True   c009fed2-6c5-4668-bb60-949dd1b6b3ed        TAČR                3  3f7dcd22-a4a2-11ed-b9df-0242ac120004        Tech
1   868a08ba-1dae-467d-b1a3-24503d5d497d              Ekologické baterie  2024-02-10T08:38:52.039022  2026-07-29T08:38:52.039027      True  a7b8b28f-37f4-422f-84bd-2ab9f31a2f50      EUREKA                6  3e9507a9-e9aa-4fcb-b273-4fcf4ea6e16d       Green
2   35d19327-c525-430b-a0e0-69186f682dde        Vodíkové palivové články  2024-03-31T08:38:52.039066  2026-11-06T08:38:52.039069      True  8396657f-934a-472a-b3e2-66535313426d       H2020                4  2d9dcd22-a4a2-11ed-b9df-0

<b>Vytvoření grafů</b>

In [152]:
with open("resultFake.json", "r") as file:
    data = json.load(file)

createGraphs(data)